In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
import tqdm

In [2]:
matplotlib.rcParams.update({
    'font.family': 'Arial',
    'font.size': 15,
    'axes.labelsize': 20,
    'axes.labelpad': 15,
    'axes.titlesize': 20,
    'axes.titlepad': 15,
})

In [3]:
rng = np.random.RandomState(4)
n_samples = 10
X_train = rng.uniform(0, 5, n_samples).reshape(-1, 1)
y_train = np.sin((X_train[:, 0] - 2.5) ** 2)

In [5]:
import torch
import torch.nn as nn

In [6]:
X = torch.from_numpy(X_train)
y = torch.from_numpy(y_train)

In [7]:
class RBF:
    def __init__(self, constant=1.0, length_scale=1.0):
        self.constant = constant
        self.length_scale = length_scale
    
    def __call__(self, X1, X2):
        dist = torch.cdist(X1, X2) ** 2 / (self.length_scale ** 2)
        kernel= (self.constant ** 2) * torch.exp(-0.5 * dist)
        return kernel

class RBFModule(nn.Module):
    def __init__(self, constant=1.0, length_scale=1.0, c_bounds=[0.1, 10], l_bounds=[0.1, 10]):
        super(RBFModule, self).__init__()
        self.constant = nn.Parameter(torch.Tensor([constant]))
        self.c_bounds = c_bounds
        self.length_scale = nn.Parameter(torch.Tensor([length_scale]))
        self.l_bounds = l_bounds
    
    def forward(self, X1, X2):
        dist = torch.cdist(X1, X2) ** 2 / (self.length_scale ** 2)
        kernel= (self.constant ** 2) * torch.exp(-0.5 * dist)
        return kernel
    
    def clip(self):
        self.constant.data.clamp_(min=self.c_bounds[0], max=self.c_bounds[1])
        self.length_scale.data.clamp_(min=self.l_bounds[0], max=self.l_bounds[1])

In [8]:
class GaussianProcess:
    def __init__(self, kernel, eps=1e-10):
        self.kernel = kernel
        self.eps = eps
    
    def fit(self, X, y):
        self.X_ = X.clone()
        self.y_ = y.clone()
        K = self.kernel(X, X)
        K[np.diag_indices(len(X))] += self.eps
        self.L = torch.linalg.cholesky(K)
        self.alpha_ = torch.cholesky_solve(y.unsqueeze(1), self.L)
    
    def predict(self, X):
        K_t = self.kernel(X, self.X_)
        y_mean = K_t.mm(self.alpha_).squeeze()
        v = torch.triangular_solve(K_t.T, self.L, upper=False)[0]
        y_cov = self.kernel(X, X) - v.T.mm(v)
        y_std = torch.sqrt(y_cov.diag())
        return y_mean, y_cov, y_std
    
    def sample_y(self, X, n_samples):
        y_mean, y_cov, y_std = self.predict(X)
        y_cov[np.diag_indices_from(y_cov)] += self.eps
        L = torch.linalg.cholesky(y_cov)
        z = torch.randn(len(X), n_samples, dtype=X.dtype)
        samples = L.mm(z) + y_mean.unsqueeze(1)
        return samples
    
    def log_marginal_likelihood(self):
        X_train = self.X_
        K = self.kernel(X_train, X_train)
        K[np.diag_indices_from(K)] += self.eps
        L = torch.linalg.cholesky(K)
        y_train = self.y_.unsqueeze(1)
        alpha = torch.cholesky_solve(y_train, L)
        log_likelihood = -0.5 * y_train.T.mm(alpha) - torch.log(torch.diag(L)).sum() - K.shape[0] / 2 * torch.log(torch.Tensor([2*torch.pi]))
        return log_likelihood

In [9]:
rbf = RBF(0.5, 0.2)
gp = GaussianProcess(rbf)

In [10]:
gp.fit(X, y)

In [11]:
gp.log_marginal_likelihood()

tensor([[-0.7624]], dtype=torch.float64)

In [12]:
X_test = torch.linspace(0, 5, 100).double().reshape(-1, 1)

In [13]:
m, cov, std = gp.predict(X_test)

In [14]:
samples = gp.sample_y(X_test, 3)

In [16]:
plt.figure(figsize=(14, 6))

plt.plot(X_test[:, 0], m, color='k', label='mean')
plt.fill_between(X_test[:, 0], m-std, m+std, alpha=0.1, color='k', label="$\pm$ 1 std.")
plt.plot(X_test[:, 0], samples, '--', label=['sample #1','sample #2','sample #3'])
plt.scatter(X[:, 0], y, c='r', s=80, label='obs.')
plt.ylim([-1.5, 1.5])
plt.xlim([0, 5])
plt.legend(bbox_to_anchor=(1.2, 0.75), loc=1)
plt.xlabel('x')
plt.ylabel('y')
plt.tight_layout()
plt.savefig('gaussian-process.png')